In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Chargement des données
df = pd.read_csv("Financial_inclusion_dataset.csv")
df.columns = [
    "pays", "année", "identifiant_unique", "compte_bancaire", "type_localisation",
    "accès_téléphone", "taille_foyer", "âge", "sexe", "relation_chef_famille",
    "statut_matrimonial", "niveau_éducation", "type_emploi"
]

# Suppression de colonnes inutiles
df = df.drop(columns=["identifiant_unique", "année", "pays", "relation_chef_famille"])

# Initialisation des encodeurs
label_encoders = {}

# Encodage des variables catégorielles sauf la cible
for col in df.select_dtypes(include=['object']):
    if col != 'type_emploi':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

# Séparation des features et de la cible
X = df.drop(columns=['type_emploi'])
Y = df['type_emploi']

# Split des données
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Modèle Random Forest
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, Y_train)

# Prédiction et évaluation
Y_pred = model.predict(X_test)
print("Rapport de classification :")
print(classification_report(Y_test, Y_pred))

# Sauvegarde du modèle
joblib.dump(model, "modelee.pkl")  # ou modele.joblib si tu préfères

# Sauvegarde des encodeurs individuellement
for col, le in label_encoders.items():
    joblib.dump(le, f"label_encoder_{col}.joblib")

# Ou : tout sauvegarder dans un seul fichier
joblib.dump(label_encoders, "label_encoders.joblib")

print("✅ Modèle et encodeurs sauvegardés avec succès.")


Rapport de classification :
                              precision    recall  f1-score   support

  Dont Know/Refuse to answer       0.00      0.00      0.00        24
         Farming and Fishing       0.43      0.48      0.45      1115
Formally employed Government       0.25      0.21      0.23        70
   Formally employed Private       0.28      0.28      0.28       212
        Government Dependent       0.02      0.02      0.02        49
         Informally employed       0.31      0.30      0.31      1105
                   No Income       0.21      0.16      0.18       127
                Other Income       0.11      0.07      0.08       223
        Remittance Dependent       0.28      0.26      0.27       501
               Self employed       0.45      0.46      0.46      1279

                    accuracy                           0.36      4705
                   macro avg       0.23      0.23      0.23      4705
                weighted avg       0.35      0.36      0.36 